In [1]:
import os

from collections import Counter
from collections import defaultdict

In [2]:
class Folder(object): 
    
    def __init__(self, root, subject_substr=""): 
        
        if "\\" in root: 
            raise Exception("Please replace the backslashes in your path with forwardslashes.")
        self.root = root
        self.subfolders = self.get_subjects(subject_substr)
        
        self.walk_history = []
        self.walk_history.append(self.subfolders)

        self.walk_history_lost = []
        self.walk_history_lost.append([])
        
     
    def get_folders_files(self, path):
        for root, folders, files in os.walk(path):
            break   
        
        return folders, files
        
    def get_subjects(self, subject_substr):         
        subjects, _ = self.get_folders_files(self.root)
        
        return [s for s in subjects if subject_substr in s]

    def down(self, equals=[], contains=[], does_not_contain=[], print_absent=False): 
        if not isinstance(equals, list) or not isinstance(contains, list) or not isinstance(does_not_contain, list):
            raise Exception("Please enter a list with strings instead of a string.")
        
        print(f"\nSubfolder equals {str(equals)},   contains {str(contains)},   does not contain {str(does_not_contain)}")
        subfolders_present = []
        subfolders_absent = []
        
        for subfolder in self.subfolders: 
            path = os.path.join(self.root, subfolder)
            
            folders, files = self.get_folders_files(path)
            
            for string in equals: 
                folders = [f for f in folders if string.lower() == f.lower()]
            
            for string in contains: 
                folders = [f for f in folders if string.lower() in f.lower()]
            
            for string in does_not_contain: 
                folders = [f for f in folders if string.lower() not in f.lower()]
            
            if len(folders) == 0: 
                subfolders_absent.append(subfolder)
                
            elif len(folders) == 1: 
                subfolders_present.append(os.path.join(subfolder, folders[0]))
                
            elif len(folders) > 1: 
                print(f"\n{subfolder} has multiple subfolders with the specified name:")
                print(folders)
                subfolders_present.append(os.path.join(subfolder, folders[0]))
        
        if len(subfolders_present) == 0:
            print("\nNo folders contained the specified subfolders, therefore this step is not executed.")
        
        else:    
            print(f"\n{len(subfolders_present)} of {len(self.subfolders)} folders contained specified subfolder.")
            if print_absent: 
                print("\nFolders without subfolder")
                for s in subfolders_absent:
                    print("\t", s)

            self.walk_history.append(subfolders_present)
            self.walk_history_lost.append(subfolders_absent)
            self.subfolders = subfolders_present
            
            print("\nPath example after step:")
            print("\t", self.subfolders[0])
        
            
    def up(self): 
        if len(self.walk_history) == 1:
            print("You can not go up as you are already at the subject folder")
        
        else: 
            self.subfolders = self.walk_history[-2]
            self.walk_history = self.walk_history[:-1]
            self.walk_history_lost = self.walk_history_lost[:-1]  
            
            print("\nPath example after step:")
            print("\t", self.subfolders[0])
    
    def sample_down(self, i=0): 
        subfolder = self.subfolders[i]
        path = os.path.join(self.root, subfolder)
            
        folders, files = self.get_folders_files(path)
        print("\nCurrent path depth:", subfolder)
        print("Subfolders:")
        for s in folders: 
            print("\t", s)
    
    def current_path(self):
        return self.subfolders[0]
            
    def number_of_files(self, print_dict=False):
        
        dct = defaultdict(list)
        
        for subfolder in self.subfolders: 
            path = os.path.join(self.root, subfolder)
            _, files = self.get_folders_files(path)
            length = len(files)
            
            dct[str(length)].append(subfolder)
        
        for l, f in dct.items():
            print(f"\n{str(len(f))} folders contain {l} files each.")
            
        if print_dict: 
            for key, value in dct.items(): 
                print(f"Folders with {key} files:")
                for s in value: 
                    print("\t", s)
            



In [3]:
import pydicom
import numpy as np

def save_3d_images(folder_class, save_to):
    for string in sorted(folder_class.subfolders):
        # get subject name and path
        subject, loc = string.split("/", 1)
        path = os.path.join(folder_class.root, string)
        
        # get all files in folder
        for root, folders, files in os.walk(path):
            break  
        files = sorted(files)
        
        # create 3d image
        three_d = []
        for image in files:
            if "DICOMDIR" not in image:
                ds = pydicom.dcmread(os.path.join(path, image))
                pix = ds.pixel_array
                three_d.append(pix)

        if len(three_d) != 0: 
            three_d = np.dstack(three_d)

            # create subfolder if not existent
            if not os.path.exists(save_to):
                os.makedirs(save_to)

            # save np array
            with open(f'{save_to}/{subject}.npy', 'wb') as f:
                np.save(f, three_d)


## Carotide

In [4]:
root = "/home/ptenkaate/lood_storage/"
path = "divi/Projects/afterfifteen/data/recon"
subjects_dir = root + path

carotide = Folder(subjects_dir)

#### Onderstaande patienten hebben twee segmentaties, die verschillen, hoe kan dat?

In [5]:
carotide.down(contains=["4dflow"])


Subfolder equals [],   contains ['4dflow'],   does not contain []

AF_0056 has multiple subfolders with the specified name:
['seg_4Dflow_PCMRA_vcalukovic', 'af_27012017_1021264_11_2_wip4dflowktpcaprospective15minV4']

AF_0114 has multiple subfolders with the specified name:
['seg_4Dflow_PCMRA_vcalukovic', 'af_20012017_1024356_11_2_wip4dflowktpcaprospective15minV4']

AF_0206 has multiple subfolders with the specified name:
['af_07012017_1502256_11_2_wip4dflowktpcaprospective15minV4', 'seg_4Dflow_PCMRA_vcalukovic']

AF_0160 has multiple subfolders with the specified name:
['af_17032017_0852211_13_2_wip4dflowktpcaprospective15minV4', 'seg_4Dflow_PCMRA_vcalukovic']

AF_0012 has multiple subfolders with the specified name:
['seg_4Dflow_PCMRA_vcalukovic', 'af_07012017_1027083_11_2_wip4dflowktpcaprospective15minV4']

AF_0154 has multiple subfolders with the specified name:
['seg_4Dflow_PCMRA_vcalukovic', 'af_20012017_0929551_11_2_wip4dflowktpcaprospective15minV4']

AF_0155 has multiple subfo

In [6]:
carotide.up()
carotide.down(contains=["wip4dflow"])


Path example after step:
	 AF_0292

Subfolder equals [],   contains ['wip4dflow'],   does not contain []

234 of 251 folders contained specified subfolder.

Path example after step:
	 AF_0292/af_28042017_0946492_11_2_wip4dflowktpcaprospective15minV4


In [7]:
carotide.down(contains=["dicom"])


Subfolder equals [],   contains ['dicom'],   does not contain []

234 of 234 folders contained specified subfolder.

Path example after step:
	 AF_0292/af_28042017_0946492_11_2_wip4dflowktpcaprospective15minV4/DICOM


In [8]:
carotide.down(equals=["pc_mra"])


Subfolder equals ['pc_mra'],   contains [],   does not contain []

234 of 234 folders contained specified subfolder.

Path example after step:
	 AF_0292/af_28042017_0946492_11_2_wip4dflowktpcaprospective15minV4/DICOM/PC_MRA


In [9]:
save_to = "/scratch/ptenkaate/Data/Carotide/pcmra"
save_3d_images(carotide, save_to)

In [10]:
carotide.number_of_files()


234 folders contain 31 files each.


In [11]:
carotide.up()
carotide.down(contains=["mask_right"], does_not_contain=["old"])
carotide.number_of_files()


Path example after step:
	 AF_0292/af_28042017_0946492_11_2_wip4dflowktpcaprospective15minV4/DICOM

Subfolder equals [],   contains ['mask_right'],   does not contain ['old']

71 of 234 folders contained specified subfolder.

Path example after step:
	 AF_0013/af_31032017_0842505_11_2_wip4dflowktpcaprospective15minV4/DICOM/Mask_Right

71 folders contain 31 files each.


In [12]:
save_to = "/scratch/ptenkaate/Data/Carotide/mask_right"
save_3d_images(carotide, save_to)

In [13]:
carotide.up()
carotide.down(contains=["mask_left"], does_not_contain=["old"])
carotide.number_of_files()


Path example after step:
	 AF_0292/af_28042017_0946492_11_2_wip4dflowktpcaprospective15minV4/DICOM

Subfolder equals [],   contains ['mask_left'],   does not contain ['old']

71 of 234 folders contained specified subfolder.

Path example after step:
	 AF_0013/af_31032017_0842505_11_2_wip4dflowktpcaprospective15minV4/DICOM/Mask_Left

71 folders contain 31 files each.


In [14]:
save_to = "/scratch/ptenkaate/Data/Carotide/mask_left"
save_3d_images(carotide, save_to)

In [15]:
carotide.up()
carotide.down(equals=["M"])
carotide.number_of_files()


Path example after step:
	 AF_0292/af_28042017_0946492_11_2_wip4dflowktpcaprospective15minV4/DICOM

Subfolder equals ['M'],   contains [],   does not contain []

234 of 234 folders contained specified subfolder.

Path example after step:
	 AF_0292/af_28042017_0946492_11_2_wip4dflowktpcaprospective15minV4/DICOM/M

234 folders contain 744 files each.


## Aorta Volunteers

In [16]:
root = "/home/ptenkaate/lood_storage/"
path = "divi/Projects/4dflow/4D-flow Volunteers/Aorta"
subjects_dir = root + path

aorta_v = Folder(subjects_dir)

In [17]:
aorta_v.down(contains=["_pca"], print_absent=True)


Subfolder equals [],   contains ['_pca'],   does not contain []

30 of 34 folders contained specified subfolder.

Folders without subfolder
	 17-03-03, Milo
	 17-11-01_Quintes_student
	 results
	 16-11-18_Male24

Path example after step:
	 16-03-18_Sanjay2_kt-pca_done2/te_18032016_1137394_7_2_wip4dflowktpcaretroV4_one_pca


In [18]:
aorta_v.down(equals=["dicom"])


Subfolder equals ['dicom'],   contains [],   does not contain []

30 of 30 folders contained specified subfolder.

Path example after step:
	 16-03-18_Sanjay2_kt-pca_done2/te_18032016_1137394_7_2_wip4dflowktpcaretroV4_one_pca/DICOM


#### Waarom hebben 2 van onderstaande volunteers meer slices in de 3d opname? Zijn de dimensies per slice hier anders?

In [19]:
# aorta_v.up()
aorta_v.down(equals=["mask"])
aorta_v.number_of_files()


Subfolder equals ['mask'],   contains [],   does not contain []

26 of 30 folders contained specified subfolder.

Path example after step:
	 16-03-18_Sanjay2_kt-pca_done2/te_18032016_1137394_7_2_wip4dflowktpcaretroV4_one_pca/DICOM/mask

24 folders contain 24 files each.

2 folders contain 30 files each.


In [20]:
save_to = "/scratch/ptenkaate/Data/Aorta Volunteers/mask"
save_3d_images(aorta_v, save_to)

In [21]:
aorta_v.up()
aorta_v.down(equals=["M"])
aorta_v.number_of_files()


Path example after step:
	 16-03-18_Sanjay2_kt-pca_done2/te_18032016_1137394_7_2_wip4dflowktpcaretroV4_one_pca/DICOM

Subfolder equals ['M'],   contains [],   does not contain []

30 of 30 folders contained specified subfolder.

Path example after step:
	 16-03-18_Sanjay2_kt-pca_done2/te_18032016_1137394_7_2_wip4dflowktpcaretroV4_one_pca/DICOM/M

28 folders contain 1152 files each.

2 folders contain 1440 files each.


In [22]:
aorta_v.up()
aorta_v.up()
aorta_v.up()
aorta_v.down(contains=["_sense"], does_not_contain=["dont_use"])


Path example after step:
	 16-03-18_Sanjay2_kt-pca_done2/te_18032016_1137394_7_2_wip4dflowktpcaretroV4_one_pca/DICOM

Path example after step:
	 16-03-18_Sanjay2_kt-pca_done2/te_18032016_1137394_7_2_wip4dflowktpcaretroV4_one_pca

Path example after step:
	 16-03-18_Sanjay2_kt-pca_done2

Subfolder equals [],   contains ['_sense'],   does not contain ['dont_use']

30 of 34 folders contained specified subfolder.

Path example after step:
	 16-03-18_Sanjay2_kt-pca_done2/te_18032016_1137394_7_2_wip4dflowktpcaretroV4_one_sense


In [23]:
aorta_v.down(contains=["dicom"])
# aorta_v.sample_down()


Subfolder equals [],   contains ['dicom'],   does not contain []

30 of 30 folders contained specified subfolder.

Path example after step:
	 16-03-18_Sanjay2_kt-pca_done2/te_18032016_1137394_7_2_wip4dflowktpcaretroV4_one_sense/DICOM


In [24]:
aorta_v.down(contains=["pc_mra"])
aorta_v.number_of_files()


Subfolder equals [],   contains ['pc_mra'],   does not contain []

30 of 30 folders contained specified subfolder.

Path example after step:
	 16-03-18_Sanjay2_kt-pca_done2/te_18032016_1137394_7_2_wip4dflowktpcaretroV4_one_sense/DICOM/PC_MRA

28 folders contain 24 files each.

2 folders contain 30 files each.


In [25]:
save_to = "/scratch/ptenkaate/Data/Aorta Volunteers/mask"
save_3d_images(aorta_v, save_to)

## Aorta BaV

In [26]:
root = "/home/ptenkaate/lood_storage/"
path = "divi/Projects/4dflow/4D-flow BAV"
subjects_dir = root + path
print(subjects_dir)

aorta_b = Folder(subjects_dir, subject_substr="done")

/home/ptenkaate/lood_storage/divi/Projects/4dflow/4D-flow BAV


In [27]:
aorta_b.down(contains=["_PCA"]) 


Subfolder equals [],   contains ['_PCA'],   does not contain []

23 of 23 folders contained specified subfolder.

Path example after step:
	 16-03-23_101_done/ru_23032016_1404331_15_2_wip4dflowktpcaretroV4_PCA


In [28]:
aorta_b.down(contains=["dicom"])


Subfolder equals [],   contains ['dicom'],   does not contain []

23 of 23 folders contained specified subfolder.

Path example after step:
	 16-03-23_101_done/ru_23032016_1404331_15_2_wip4dflowktpcaretroV4_PCA/DICOM


#### aorta_122_withgap heeft een eerste slice met segmentatie, die in Aorta zelf mist. Bij 121 is er ook enkel een verschil in de eerste slice.

In [29]:
aorta_b.down(contains=["aorta"],print_absent=True)


Subfolder equals [],   contains ['aorta'],   does not contain []

16-03-23_101_done/ru_23032016_1404331_15_2_wip4dflowktpcaretroV4_PCA/DICOM has multiple subfolders with the specified name:
['Aorta_seg-sense', 'Aorta']

16-08-26_121_done1/ta_26082016_1145135_18_2_wip4dflowktpcaretroV4_PCA/DICOM has multiple subfolders with the specified name:
['Aorta_121', 'Aorta']

16-09-02_122_done1/ha_02092016_1121065_4_2_wip4dflowktpcaretroV4_PCA/DICOM has multiple subfolders with the specified name:
['Aorta_122_withgap', 'Aorta']

23 of 23 folders contained specified subfolder.

Folders without subfolder

Path example after step:
	 16-03-23_101_done/ru_23032016_1404331_15_2_wip4dflowktpcaretroV4_PCA/DICOM/Aorta_seg-sense


In [30]:
aorta_b.up()
aorta_b.down(equals=["Aorta"])


Path example after step:
	 16-03-23_101_done/ru_23032016_1404331_15_2_wip4dflowktpcaretroV4_PCA/DICOM

Subfolder equals ['Aorta'],   contains [],   does not contain []

23 of 23 folders contained specified subfolder.

Path example after step:
	 16-03-23_101_done/ru_23032016_1404331_15_2_wip4dflowktpcaretroV4_PCA/DICOM/Aorta


In [31]:
save_to = "/scratch/ptenkaate/Data/Aorta BaV/mask"
save_3d_images(aorta_b, save_to)

#### waarom verschilt het aantal slices? Is de dimensie per slice wel gelijk?

In [32]:
aorta_b.number_of_files()


2 folders contain 25 files each.

6 folders contain 30 files each.

1 folders contain 26 files each.

13 folders contain 24 files each.

1 folders contain 36 files each.


In [33]:
aorta_b.up()
aorta_b.down(equals=["M"], print_absent=True)


Path example after step:
	 16-03-23_101_done/ru_23032016_1404331_15_2_wip4dflowktpcaretroV4_PCA/DICOM

Subfolder equals ['M'],   contains [],   does not contain []

21 of 23 folders contained specified subfolder.

Folders without subfolder
	 16-03-23_101_done/ru_23032016_1404331_15_2_wip4dflowktpcaretroV4_PCA/DICOM
	 16-08-03_100_done1/ja_03082016_1050555_18_2_wip4dflowktpcaretroV4_PCA/DICOM

Path example after step:
	 16-08-19_119_done1/ko_19082016_1549079_13_2_wip4dflowktpcaretroV4_PCA/DICOM/M


#### hier komt het aantal wel overeen met pc_mra. Dus aantal tijdsstappen is hetzelfde

In [34]:
aorta_b.number_of_files()


6 folders contain 1440 files each.

1 folders contain 1248 files each.

12 folders contain 1152 files each.

1 folders contain 1728 files each.

1 folders contain 1200 files each.


In [35]:
aorta_b.up()
aorta_b.up()
aorta_b.up()
aorta_b.down(contains=["_sense"])


Path example after step:
	 16-03-23_101_done/ru_23032016_1404331_15_2_wip4dflowktpcaretroV4_PCA/DICOM

Path example after step:
	 16-03-23_101_done/ru_23032016_1404331_15_2_wip4dflowktpcaretroV4_PCA

Path example after step:
	 16-03-23_101_done

Subfolder equals [],   contains ['_sense'],   does not contain []

23 of 23 folders contained specified subfolder.

Path example after step:
	 16-03-23_101_done/ru_23032016_1404331_15_2_wip4dflowktpcaretroV4_Sense


In [36]:
aorta_b.down(contains=["dicom"])
# aorta_b.sample_down(3)


Subfolder equals [],   contains ['dicom'],   does not contain []

23 of 23 folders contained specified subfolder.

Path example after step:
	 16-03-23_101_done/ru_23032016_1404331_15_2_wip4dflowktpcaretroV4_Sense/DICOM


In [37]:
aorta_b.down(contains=["pc_mra"])


Subfolder equals [],   contains ['pc_mra'],   does not contain []

16-07-15_114_done1/be_15072016_1037278_16_2_wip4dflowktpcaretroV4_sense/DICOM has multiple subfolders with the specified name:
['PC_MRA', 'PC_MRA_old']

23 of 23 folders contained specified subfolder.

Path example after step:
	 16-03-23_101_done/ru_23032016_1404331_15_2_wip4dflowktpcaretroV4_Sense/DICOM/PC_MRA


In [38]:
aorta_b.number_of_files()


2 folders contain 25 files each.

6 folders contain 30 files each.

1 folders contain 26 files each.

13 folders contain 24 files each.

1 folders contain 36 files each.


In [39]:
save_to = "/scratch/ptenkaate/Data/Aorta BaV/pcmra"
save_3d_images(aorta_b, save_to)

## Aorta CoA

In [40]:
root = "/home/ptenkaate/lood_storage/"
path = "divi/Projects/4dflow/4D-flow CoA"
subjects_dir = root + path
print(subjects_dir)

aorta_c = Folder(subjects_dir, subject_substr="done")
# aorta_c.sample_down(2)

/home/ptenkaate/lood_storage/divi/Projects/4dflow/4D-flow CoA


In [41]:
aorta_c.down(contains=["_pca"])
# aorta_c.sample_down()


Subfolder equals [],   contains ['_pca'],   does not contain []

43 of 43 folders contained specified subfolder.

Path example after step:
	 16-04-01_203_done/wi_01042016_1123584_14_2_wip4dflowktpcaretroV4_pca


In [42]:
aorta_c.down(contains=["dicom"])


Subfolder equals [],   contains ['dicom'],   does not contain []

43 of 43 folders contained specified subfolder.

Path example after step:
	 16-04-01_203_done/wi_01042016_1123584_14_2_wip4dflowktpcaretroV4_pca/DICOM


#### weer meerdere segmentaties, hoe behandel ik deze?

In [43]:
aorta_c.down(contains=["aorta"])


Subfolder equals [],   contains ['aorta'],   does not contain []

16-03-11_204_done/vVe_11032016_1149331_14_2_wip4dflowktpcaretroV4_PCA/DICOM has multiple subfolders with the specified name:
['Aorta_metgat', 'Aorta']

16-06-17_220_done/sm_17062016_1223277_20_2_4dflowktpcaretroV4_Pca/DICOM has multiple subfolders with the specified name:
['Aorta_oud', 'Aorta']

16-09-14_242_done1/ro_14092016_1154409_17_2_wip4dflowktpcaretroV4_PCA/DICOM has multiple subfolders with the specified name:
['Aorta_empty', 'Aorta']

43 of 43 folders contained specified subfolder.

Path example after step:
	 16-04-01_203_done/wi_01042016_1123584_14_2_wip4dflowktpcaretroV4_pca/DICOM/Aorta


In [44]:
aorta_c.up()
aorta_c.down(equals=["Aorta"])


Path example after step:
	 16-04-01_203_done/wi_01042016_1123584_14_2_wip4dflowktpcaretroV4_pca/DICOM

Subfolder equals ['Aorta'],   contains [],   does not contain []

43 of 43 folders contained specified subfolder.

Path example after step:
	 16-04-01_203_done/wi_01042016_1123584_14_2_wip4dflowktpcaretroV4_pca/DICOM/Aorta


In [45]:
save_to = "/scratch/ptenkaate/Data/Aorta CoA/mask"
save_3d_images(aorta_c, save_to)

#### verschil in aantal slices

In [46]:
aorta_c.number_of_files()


28 folders contain 24 files each.

13 folders contain 30 files each.

1 folders contain 28 files each.

1 folders contain 26 files each.


In [47]:
aorta_c.up()
aorta_c.down(equals=["M"])


Path example after step:
	 16-04-01_203_done/wi_01042016_1123584_14_2_wip4dflowktpcaretroV4_pca/DICOM

Subfolder equals ['M'],   contains [],   does not contain []

43 of 43 folders contained specified subfolder.

Path example after step:
	 16-04-01_203_done/wi_01042016_1123584_14_2_wip4dflowktpcaretroV4_pca/DICOM/M


In [48]:
aorta_c.number_of_files()


28 folders contain 1152 files each.

13 folders contain 1440 files each.

1 folders contain 1344 files each.

1 folders contain 1248 files each.


In [49]:
aorta_c.up()
aorta_c.up()
aorta_c.up()


Path example after step:
	 16-04-01_203_done/wi_01042016_1123584_14_2_wip4dflowktpcaretroV4_pca/DICOM

Path example after step:
	 16-04-01_203_done/wi_01042016_1123584_14_2_wip4dflowktpcaretroV4_pca

Path example after step:
	 16-04-01_203_done


In [50]:
aorta_c.down(contains=["_sense"])


Subfolder equals [],   contains ['_sense'],   does not contain []

43 of 43 folders contained specified subfolder.

Path example after step:
	 16-04-01_203_done/wi_01042016_1123584_14_2_wip4dflowktpcaretroV4_sense


In [51]:
aorta_c.down(contains=["dicom"])


Subfolder equals [],   contains ['dicom'],   does not contain []

43 of 43 folders contained specified subfolder.

Path example after step:
	 16-04-01_203_done/wi_01042016_1123584_14_2_wip4dflowktpcaretroV4_sense/DICOM


In [52]:
aorta_c.down(contains=["pc_mra"])


Subfolder equals [],   contains ['pc_mra'],   does not contain []

43 of 43 folders contained specified subfolder.

Path example after step:
	 16-04-01_203_done/wi_01042016_1123584_14_2_wip4dflowktpcaretroV4_sense/DICOM/PC_MRA


In [53]:
aorta_c.number_of_files()


28 folders contain 24 files each.

13 folders contain 30 files each.

1 folders contain 28 files each.

1 folders contain 26 files each.


In [54]:
save_to = "/scratch/ptenkaate/Data/Aorta CoA/pcmra"
save_3d_images(aorta_c, save_to)

## Aorta RESV

In [55]:
root = "/home/ptenkaate/lood_storage/"
path = "divi/Projects/resvcue_marfan/analysis/RESVcue Marfan"
subjects_dir = root + path
print(subjects_dir)

aorta_r = Folder(subjects_dir, subject_substr="RESV")

/home/ptenkaate/lood_storage/divi/Projects/resvcue_marfan/analysis/RESVcue Marfan


In [56]:
aorta_r.down()


Subfolder equals [],   contains [],   does not contain []

RESV_010 has multiple subfolders with the specified name:
['V1 05-10-2019', 'V2 10-10-2020']

60 of 62 folders contained specified subfolder.

Path example after step:
	 RESV_015/V1 04-11-2019


In [57]:
aorta_r.down(contains=["segmentatie"], print_absent=True)


Subfolder equals [],   contains ['segmentatie'],   does not contain []

57 of 60 folders contained specified subfolder.

Folders without subfolder
	 RESV_022/V1 04-01-2020
	 RESV_019/V1 03-07-2019
	 RESV_024/V1 21-10-2019

Path example after step:
	 RESV_015/V1 04-11-2019/Segmentatie


#### weer verschil in aantal slices. Zijn dimensies hetzelfde?

In [58]:
aorta_r.number_of_files()


15 folders contain 30 files each.

40 folders contain 24 files each.

2 folders contain 0 files each.


In [59]:
save_to = "/scratch/ptenkaate/Data/Aorta Resque/mask"
save_3d_images(aorta_r, save_to)

In [60]:
aorta_r.up()
aorta_r.down(contains=["4d flow"], print_absent=True)


Path example after step:
	 RESV_015/V1 04-11-2019

Subfolder equals [],   contains ['4d flow'],   does not contain []

RESV_020/V1 24-08-2019 has multiple subfolders with the specified name:
['4D flow recon out - opnieuw geroncstrueerd', '4D flow recon out']

57 of 60 folders contained specified subfolder.

Folders without subfolder
	 RESV_022/V1 04-01-2020
	 RESV_019/V1 03-07-2019
	 RESV_024/V1 21-10-2019

Path example after step:
	 RESV_015/V1 04-11-2019/4D flow recon out


In [61]:
aorta_r.down(print_absent=True)


Subfolder equals [],   contains [],   does not contain []

RESV_205/V1 07-11-2020/4D flow recon out has multiple subfolders with the specified name:
['DICOM', 'DICOM_PCMRA']

RESV_111/V1 04-11-2020/4D flow recon out has multiple subfolders with the specified name:
['DICOM', 'DICOM_PCMRA']

RESV_204/V1 24-10-2020/4D flow recon out has multiple subfolders with the specified name:
['DICOM', 'DICOM_PCMRA']

56 of 57 folders contained specified subfolder.

Folders without subfolder
	 RESV_010/V1 05-10-2019/4D flow recon out

Path example after step:
	 RESV_015/V1 04-11-2019/4D flow recon out/re_04112019_1516-out


In [62]:
aorta_r.subfolders

['RESV_015/V1 04-11-2019/4D flow recon out/re_04112019_1516-out',
 'RESV_026/V1 08-02-2020/4D flow recon out/7890_20200208_095146',
 'RESV_306/V1 22-02-2020/4D flow recon out/RESV306_20200222_135330',
 'RESV_110/V1 24-10-2020/4D flow recon out/RESV110_20201024_120328',
 'RESV_308/V1 04-07-2020/4D flow recon out/RESV308_20200704_102458',
 'RESV_104/V1 29-08-2020/4D flow recon out/RESV104_20200829_120156',
 'RESV_312/V1 03-10-2020/4D flow recon out/RESV312_20201003_135713',
 'RESV_203/V1 15-08-2020/4D flow recon out/RESV203_20200815_120400',
 'RESV_201/V1 20-06-2020/4D flow recon out/RESV201_20200620_122916',
 'RESV_600/V1 10-10-2020/4D flow recon out/RESV600_20201010_092904',
 'RESV_105/V1 15-08-2020/4D flow recon out/RESV105_20200815_101923',
 'RESV_310/V1 29-08-2020/4D flow recon out/RESV310_20200829_093535',
 'RESV_027/V1 22-02-2020/4D flow recon out/RESV027_20200222_090235',
 'RESV_309/V1 12-09-2020/4D flow recon out/RESV309_20200912_115841',
 'RESV_017/V1 11-07-2020/4D flow recon o

In [63]:
aorta_r.down(contains=["dicom_pcmra"], print_absent=True)


Subfolder equals [],   contains ['dicom_pcmra'],   does not contain []

49 of 56 folders contained specified subfolder.

Folders without subfolder
	 RESV_012/V1 24-08-2019/4D flow recon out/recon_out_RESVcue_20190824_3
	 RESV_014/V1 20-06-2019/4D flow recon out/recon_out_PROUD_4Dflow
	 RESV_205/V1 07-11-2020/4D flow recon out/DICOM
	 RESV_111/V1 04-11-2020/4D flow recon out/DICOM
	 RESV_011/V1 24-08-2019/4D flow recon out/recon_out_RESVcue_20190824_4
	 RESV_006/V1 24-08-2019/4D flow recon out/recon_out_RESVcue_20190824_1
	 RESV_204/V1 24-10-2020/4D flow recon out/DICOM

Path example after step:
	 RESV_015/V1 04-11-2019/4D flow recon out/re_04112019_1516-out/DICOM_PCMRA


In [64]:
aorta_r.down(contains=["pcmra"], print_absent=True)


Subfolder equals [],   contains ['pcmra'],   does not contain []

49 of 49 folders contained specified subfolder.

Folders without subfolder

Path example after step:
	 RESV_015/V1 04-11-2019/4D flow recon out/re_04112019_1516-out/DICOM_PCMRA/PCMRA


#### Dit waren de getallen voor segmentatie:

15 folders contain 30 files each.

40 folders contain 24 files each.

2 folders contain 0 files each.

#### Komst niet overeen met de PCMRA, wat gebeurt er.

In [65]:
aorta_r.number_of_files()


8 folders contain 61 files each.

16 folders contain 49 files each.

17 folders contain 24 files each.

8 folders contain 30 files each.


In [66]:
save_to = "/scratch/ptenkaate/Data/Aorta Resque/pcmra"
save_3d_images(aorta_r, save_to)

## Heart LUMC

In [67]:
root = "/home/ptenkaate/lood_storage/"
path = "divi/Projects/4dflowimpulse/Pilot study with LUMC/Scans"
subjects_dir = root + path
print(subjects_dir)

heart_l = Folder(subjects_dir, subject_substr="PROUD")

/home/ptenkaate/lood_storage/divi/Projects/4dflowimpulse/Pilot study with LUMC/Scans


In [68]:
heart_l.down(contains=["segm"])


Subfolder equals [],   contains ['segm'],   does not contain []

12 of 12 folders contained specified subfolder.

Path example after step:
	 20181113_EPI-PROUD/SEGM


In [69]:
heart_l.down(contains=['proud'], print_absent=True)


Subfolder equals [],   contains ['proud'],   does not contain []

11 of 12 folders contained specified subfolder.

Folders without subfolder
	 20181210_EPI-PROUD/PROUD SEGM

Path example after step:
	 20181113_EPI-PROUD/SEGM/PROUD SEGM


In [70]:
heart_l.number_of_files()


9 folders contain 35 files each.

1 folders contain 40 files each.

1 folders contain 37 files each.


In [71]:
save_to = "/scratch/ptenkaate/Data/Heart LUMC/mask"
save_3d_images(heart_l, save_to)

In [72]:
heart_l.up()
heart_l.up()


Path example after step:
	 20181113_EPI-PROUD/SEGM

Path example after step:
	 20181113_EPI-PROUD


In [73]:
heart_l.down(contains=['4d flow'], print_absent=True)


Subfolder equals [],   contains ['4d flow'],   does not contain []

12 of 12 folders contained specified subfolder.

Folders without subfolder

Path example after step:
	 20181113_EPI-PROUD/4D flow


In [74]:
heart_l.down(contains=["proud"])


Subfolder equals [],   contains ['proud'],   does not contain []

20181031_EPI-PROUD/4D flow has multiple subfolders with the specified name:
['recon_out_20181031_PROUD_1_noNC', 'recon_out_20181031_1_PROUD_nc100']

20181031_EPI-PROUD_2/4D flow has multiple subfolders with the specified name:
['recon_out_20181031_2_PROUD_NCpcmra', 'recon_out_20181031_2_PROUD_nc100']

20181029_2_EPI-PROUD/4D flow has multiple subfolders with the specified name:
['recon_out_20181029_2_PROUD_noNC', 'recon_out_20181029_2_PROUD_nc100']

12 of 12 folders contained specified subfolder.

Path example after step:
	 20181113_EPI-PROUD/4D flow/recon_out_20181113_PROUD_noNC


In [75]:
# heart_l.subfolders

#### Wat is het verschil tussen deze twee?

In [76]:
heart_l.down(contains=["pcmra"])


Subfolder equals [],   contains ['pcmra'],   does not contain []

20181113_EPI-PROUD/4D flow/recon_out_20181113_PROUD_noNC has multiple subfolders with the specified name:
['PCMRA', 'NC-PCMRA']

20181212_EPI-PROUD/4D flow/recon_out_20181212_PROUD has multiple subfolders with the specified name:
['PCMRA', 'NC-PCMRA']

20181128_EPI-PROUD/4D flow/recon_out_20181128_PROUD_2pcmra has multiple subfolders with the specified name:
['PCMRA', 'NC-PCMRA']

20181115_2_EPI-PROUD/4D flow/recon_out_20181115_2_PROUD_NCpcmra has multiple subfolders with the specified name:
['PCMRA', 'NC-PCMRA']

20181210_EPI-PROUD/4D flow/recon_out_20181210_1_PROUD_NCpcmra has multiple subfolders with the specified name:
['PCMRA', 'NC-PCMRA', 'NC-PCMRA_threshold150']

20181211_EPI-PROUD/4D flow/recon_out_20181211_PROUD has multiple subfolders with the specified name:
['PCMRA', 'NC-PCMRA']

12 of 12 folders contained specified subfolder.

Path example after step:
	 20181113_EPI-PROUD/4D flow/recon_out_20181113_PROUD_no

In [77]:
heart_l.down()


Subfolder equals [],   contains [],   does not contain []

12 of 12 folders contained specified subfolder.

Path example after step:
	 20181113_EPI-PROUD/4D flow/recon_out_20181113_PROUD_noNC/PCMRA/DICOM


In [78]:
heart_l.number_of_files()


10 folders contain 70 files each.

1 folders contain 80 files each.

1 folders contain 74 files each.


In [79]:
save_to = "/scratch/ptenkaate/Data/Heart LUMC/pcmra"
save_3d_images(heart_l, save_to)

## Heart Vital

In [80]:
root = "/home/ptenkaate/lood_storage/"
path = "divi/Projects/cs_4dflow/project_VitalEyeComparisonClinical/data"
subjects_dir = root + path
print(subjects_dir)

heart_v = Folder(subjects_dir)
heart_v.sample_down(2)

/home/ptenkaate/lood_storage/divi/Projects/cs_4dflow/project_VitalEyeComparisonClinical/data

Current path depth: subject_29
Subfolders:
	 recon_out_subject_29_none
	 recon_out_subject_29_RNAV
	 recon_out_subject_29_VE
	 recon_out_subject_29_noGating_nonwrap


In [81]:
heart_v.down(contains=["segm"])
heart_v.subfolders


Subfolder equals [],   contains ['segm'],   does not contain []

4 of 32 folders contained specified subfolder.

Path example after step:
	 subject_07/segm_sub07


['subject_07/segm_sub07',
 'subject_01/segm_sub01',
 'subject_13/Z_SEGM',
 'subject_02/segm_sub02']

In [82]:
heart_v.up()


Path example after step:
	 subject_17


In [83]:
heart_v.down(contains=["VE"])



Subfolder equals [],   contains ['VE'],   does not contain []

subject_30 has multiple subfolders with the specified name:
['recon_out_subject_30_VE', 'zzz_recon_out_subject_30_VE_unflipped']

32 of 32 folders contained specified subfolder.

Path example after step:
	 subject_17/recon_out_subject_17_VE


In [84]:
heart_v.down(contains=["pcmra"])



Subfolder equals [],   contains ['pcmra'],   does not contain []

32 of 32 folders contained specified subfolder.

Path example after step:
	 subject_17/recon_out_subject_17_VE/PCMRA


In [85]:
heart_v.number_of_files()


11 folders contain 63 files each.

16 folders contain 56 files each.

4 folders contain 49 files each.

1 folders contain 84 files each.
